## Multinomial Variation Autoencoder for Collaborative Filtering

I must admit that when it comes to variational autoencoders I find that there is a "*notable*" difference between the complexity of the math and that of the code (or maybe is just me that I am not a mathematician). Nonetheless, I think that speaking about VAEs and not discussing log likelihoods, Evidence Lower Bound (EBLO) and some other mathematical terms is almost like "cheating". With that in mind I will try to give some mathematical context to the Multinomial VAE (Mult-VAE) for collaborative filtering and then move to the code. Bear in mind that the whole purpose of the math below is to "smoothly" get to the loss function we will be using for this exercise. 

The references I used for the following writing are: 

* [Auto-Encoding Variational Bayes](https://arxiv.org/pdf/1312.6114.pdf): a must-read paper to understand variational autoencoders.
* [Variational Autoencoders for Collaborative Filtering](https://arxiv.org/pdf/1802.05814.pdf): the paper on which this whole repo is based.
* [Variational Autoencoders with Gluon](https://gluon.mxnet.io/chapter13_unsupervised-learning/vae-gluon.html): one of the reason why like [Gluon](https://mxnet.incubator.apache.org/api/python/docs/api/gluon/index.html) is because some of the tutorials they have on their site are just amazing. This is one example.
* [EM Demystified: An Expectation-Maximization Tutorial](https://vannevar.ece.uw.edu/techsite/papers/documents/UWEETR-2010-0002.pdf): one of the references within Gluon's tutorial is this tutorial, which I find dense, but useful when it comes to understanding some of the concepts I will discuss below.

Overall, I find these 4 references fantastic and I strongly encourage everyone that is reading these lines and is interested in variational autoencoders to give them a read. In fact, the writing here is mostly based in the Gluon tutorial, adapted to the case of VAEs for collaborative filtering. 

### The EM algorithm

The problem we face here is trying to find out the likelihood, or better, the log-likelihood $\ell(\theta)$ of a given dataset of, for example, user clicks. The way I normally think of this is that we are trying to find the underlying probability that a dataset exists, so that we can then do inference with it. A standard way of doing this is using an Expectation-Maximization (EM) algorithm. When using EM, instead of using directly the data $x$ (clicks in this example) to maximize $\ell$, we use a latent representation of $x$, $z$, to maximize a lower bound likelihood $\mathcal{L}(q,\theta)$, where $q$ is any valid probability distribution and $\theta$ are the parameters of the log-likelihood function $\ell(\theta) = \sum_i \log\left( p_{\theta}(x_i) \right)$. This can be formulated as follows: 

$$
\begin{equation*}
\begin{split}
\ell(\theta) &= \sum_i \log\left( p_{\theta}(x_i) \right) \\
& = \sum_i \log\left( \int p_{\theta}(x_i, z) dz \right)\\
&= \sum_i \log\left( \mathbb{E}_{z \sim Q} \left[ \frac {p_{\theta}(x_i, z)}{q(z)} \right]\right) \\ 
& {\ge}\underbrace{ \sum_i \mathbb{E}_{z \sim Q} \left[\log\left( \frac {p_{\theta}(x_i,z)}{q(z)} \right)\right]}_{ELBO: \mathcal{L}(q,\theta)} \hspace{5cm} (1)
\end{split} 
\end{equation*}
$$

Please go [here](https://gluon.mxnet.io/chapter13_unsupervised-learning/vae-gluon.html) and references therein, in particular equation 4.1 of the [tutorial](https://vannevar.ece.uw.edu/techsite/papers/documents/UWEETR-2010-0002.pdf) mentioned before, for a proper derivation of Eq (1), which derives from Jensen’s inequality and holds for any $q(z)$ as long as it is a valid probability distribution.

Looking at Eq (1) we can see that our job is basically to maximize ELBO $\mathcal{L}(q,\theta)$ so we maximize $\ell(\theta)$. To that aim, we **need** to chose $q(z)$ at a given iteration so that is the inferred posterior $p(z\vert x)$, i.e. at $t$-th iteration, we chose $q$ so that:  

$$
q^t(z) = p(z\vert x_i; \hat\theta^{t-1}) = \frac{p(x_i\vert z; \hat\theta^{t-1})p(z; \hat\theta^{t-1})}{\int p(x_i\vert z; \hat\theta^{t-1})p(z; \hat\theta^{t-1}) dz} \hspace{5cm} (2)
$$

Choosing that $q$ is basically the essence of the E-step. Then, in the M-step we maximize over the parameters $\theta$. 

In words, the chain of improvements through E-step and M-step can be described as follows: during the E-step we chose $q$ so that **is** the inferred posterior, i.e. Eq (1) becomes an equality. Then, during the M-step we maximize over $\theta$ so that we ensure that $\mathcal L(q^t,\theta^{t-1}) \le \mathcal L(q^t,\theta^t)$. Finally, Jensen's inequality ensures that $ \mathcal L(q^t,\theta^t) \le \ell(\theta^{t}) $. 

Altogether:

$$
\ell(\theta^{t-1}) \underset{E-step}{=} \mathcal L(q^t,\theta^{t-1}) \underset{M-step}{\le} \mathcal L(q^t,\theta^t) \underset{Jensen}{\le} \ell(\theta^{t}) \hspace{5cm} (3)
$$

### Variational Autoencoders

As mentioned before, to maximize ELBO $\mathcal{L}(q,\theta)$ we need to chose $q$ so that is the inferred posterior at $t$-th iteration during the E-step. However, for complex distribution $p_{\theta}(x|z)$, the computation of the posterior $p_{\theta}(z|x)$ is intractable. To solve this problem we can use variational inference methods. In particular, [Kingma and Welling 2014](https://arxiv.org/pdf/1312.6114.pdf) introduced a "simple" neural-network approach, *Auto-Encoding Variational Bayes*, where the variational inference/optimization task of finding the optimal $q$ becomes a matter of finding the best parameters of a neural network via backpropagation and stochastic gradient descent. 

Because from now on I want to adapt the text to the Mult-VAE encoder, let me add here a couple of lines about notation. Following [Liang et al., 2018]() I will use $u \in \{1,\dots,U\}$ to index users and $i \in \{1,\dots,I\}$ to index items. The user-by-item **binary** interaction matrix (i.e. the click matrix) is $\mathbf{X} \in \mathbb{N}^{U\times I}$ and I will use lower case $\mathbf{x}_u =[X_{u1},\dots,X_{uI}]^\top \in \mathbb{N}^I$ to refer to the click history of an individual user $u$.

Moving on and quoting **directly** the [Gluon tutorial](https://gluon.mxnet.io/chapter13_unsupervised-learning/vae-gluon.html), this is how Auto-Encoding Variational Bayes works:

1. Select a prior for latent representation of $\textbf{x}_u$, $p_{\theta}(\textbf{z}_u)$
2. Use a neural network to parameterize the distribution $p_{\theta}(\textbf{x}_u\vert \textbf{z}_u)$. Because this part of the model maps the latent variable/representation $\textbf{z}_u$ to the observed data $\textbf{x}_u$, it is referred as a "*decoder*" network. 
3. Rather than explicitly calculating the intractable posterior $p_{\theta}(\textbf{z}_u\vert \textbf{x}_u)$, we use another another neural network to parameterize the distribution $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$ as the approximate posterior. Since $q_\phi$ maps the observed data $\textbf{x}_u$ to the latent space of $\textbf{z}_u$'s, is referred as the "*encoder*" network.
4. The objective is still to maxmize ELBO $\mathcal{L}(q,\theta)$. But now instead of separately finding the optimal $\phi$ (this would be equivalent to chosing $q$ in EM) and $\theta$ like EM, we can find the parameters $\theta$ and $\phi$ jointly via standard stochastic gradient descent.

Since we have an encoder-decoder structure, we refer to this as variational auto-encoder (VAE).

Now, given the distribution $p_{\theta}(\textbf{x}_u\vert \textbf{z}_u)$ and the approximate posterior $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$, we can re-write the ELBO $\mathcal{L}(q,\theta)$ in Eq (1) in terms of $\theta$ and $\phi$ as: 

$$
\begin{split}\begin{equation*}
\begin{split}
- \mathcal L(\textbf{x}_u, \phi,\theta) & = - \mathbb{E}_{\textbf{z}_u \sim Q_\phi(\textbf{z}_u|\textbf{x}_u)} \left[\log p_{\theta}(\textbf{x}_u \vert \textbf{z}_u) + \log p_\theta(\textbf{z}_u) - \log q_\phi(\textbf{z}_u\vert \textbf{x}_u) \right] \\
& = - \mathbb{E}_{\textbf{z}_u \sim Q_\phi(\textbf{z}_u|\textbf{x}_u)} \left[\log p_{\theta}(\textbf{x}_u \vert \textbf{z}_u) \right] + D_{KL}\left[\log q_\phi(\textbf{z}_u\vert \textbf{x}_u) \| p_\theta(\textbf{z}_u)\right] \hspace{5cm} (4) 
\end{split}
\end{equation*}\end{split}
$$

Where $D_{KL}$ is the [Kullback–Leibler divergence](). See [this wikipedia page]() to understand how the expansion of $p(x,z)$ allows us to re-write the equation in that form. Also, and more importantly, note that maximizing $\mathcal L(\textbf{x}_u, \phi,\theta)$ involves minimizing $D_{KL}$, which makes sense, since $D_{KL}$ measures the dissimilarity of the approximate posterior  $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$ from the true posterior $p_{\theta}(\textbf{z}_u\vert \textbf{x}_u)$.

Coming back to the paper [Auto-Encoding Variational Bayes](), the authors described there what is often referred as the **Gaussian VAE**, where we have a Gaussian prior $p(\textbf{z}_u) \sim \mathcal N(0, I)$ and the approximate posterior is also assumed to be Gaussian $q_\phi(\textbf{z}_u\vert \textbf{x}_u) \sim \mathcal N(\mu_\phi(\textbf{x}_u), \sigma_\phi(\textbf{x}_u) I)$, where $\mu_\phi(\textbf{x}_u)$ and  $\sigma_\phi(\textbf{x}_u)$ are the outputs of some neural network, for example, the outputs of a MLP. With this set up, Kingma and Welling show that the Gaussian VAE can be trained by minimizing the negative ELBO and Eq (4) can be written as:

$$
\begin{equation*}
- \mathcal L(\textbf{x}_u, \phi,\theta) \approx \frac{1}{L} \sum_s^L \left[-\log p_{\theta}(\textbf{x}_u \vert \textbf{z}_{us}) \right] + D_{KL}\left[\log q_\phi(\textbf{z}_u\vert \textbf{x}_u) \| p_\theta(\textbf{z}_u)\right] \hspace{5cm} (5)
\end{equation*}
$$

The **Mult-VAE** uses a very similar that set up. As before, for a user $u$, the latent representation $\textbf{z}_u$ is assumed to be drawn from a standard Gaussian prior $p(\textbf{z}_u) \sim \mathcal N(0, I)$. Such representation is then transformed by a non-linear function $f_{\theta}(.)$ (e.g. a MLP) and the output is normalized via a Softmax function to produce a probability distribution over all items **$I$**, $\pi(\textbf{z}_u)$. If we then defined the total number of clicks as $N_u = \sum_i x_{ui}$, **the click history of user $u$ is assumed to be drawn from a Multinomial distribution with parameters**: 

$$
\textbf{x}_u \sim \text{Mult}(N_u, \pi(\textbf{z}_u)) \hspace{5cm} (6)
$$

Under this assumptions:

$$
\begin{equation*}
\log(p_{\theta}(\textbf{x}_u\vert \textbf{z}_u)) = \sum_i x_{ui} \log \pi_i(\textbf{z}_{u}) \hspace{5cm} (7)
\end{equation*}
$$


and therefore, "abusing notation" a bit, Eq (5) can be written as:

$$
\begin{equation*}
- \mathcal L(\textbf{x}_u, \phi,\theta) \approx - \frac{1}{L} \sum_s^L \sum_i x_{ui} \left[ \log \pi_i(\textbf{z}_{us})  \right] + D_{KL}\left[\log q_\phi(\textbf{z}_u\vert \textbf{x}_u) \| p_\theta(\textbf{z}_u)\right] \hspace{5cm} (8)
\end{equation*}
$$

Note that Eq (8) involves sampling $\textbf{z}_{us} \sim q_\phi(\textbf{z}_u\vert \textbf{x}_u)$. When sampling is involved, backpropagation is not trivial (how one would take gradients with respect to $\phi$?). This was solved by Kingma and Welling by introducing the reparameterization trick. If we remember: $q_\phi(\textbf{z}_u\vert \textbf{x}_u) \sim \mathcal N(\mu_\phi(\textbf{x}_u), \sigma_\phi(\textbf{x}_u) I)$. Therefore, Instead of sampling $\textbf{z}_u$ from that distribution, we construct $\textbf{z}_u = \mu(\textbf{x}_u) + \sigma(\textbf{x}_u) \cdot \epsilon$, where $\epsilon \sim \mathcal{N}(0,I)$, and we sample directly $\epsilon$. This way $\textbf{z}_u$ depends deterministically on $\mu(\textbf{x}_u)$ and  $\sigma(\textbf{x}_u)$ and we can compute gradients. 

In summary, given the Gaussian prior and approximate posterior discussed above, the training loss we need to compute after the reparameterization trick for the Mult-VAE is:

$$
\begin{equation*}
\mathcal L(\textbf{x}_u, \phi,\theta) = - \frac{1}{N_u}\sum_i x_{ui} \left[ \textit{log_softmax}(\textbf{z}_u) \right] + \frac{1}{2N_u}\left[-\sum_i\left(\log\sigma_i^2 + 1\right) + \sum_i\sigma_i^2 + \sum_i\mu^2_i\right] \hspace{5cm} (9)
\end{equation*}
$$

If you want to have a look to math behind the $D_{KL}$ expression, see [here]().

Before we move to the code there is one, and just one more aspect we need to discuss. In their paper, Liang et al. propose an alternative interpretation of ELBO. The authors interpret the first term in Eq (9) as the reconstruction loss, while the second can be viewed as regularization. Quoting directly from the authors: "*It is this perspective we work with because it allows us to make a trade-off that forms the crux of our method. From this perspective, it is natural to extend the ELBO by introducing a parameter $\beta$ to control the strength of
regularization*". With this formulation Eq (9) simply turns into:

$$
\begin{equation*}
\mathcal L(\textbf{x}_u, \phi,\theta) = - \frac{1}{N_u}\sum_i x_{ui} \left[ \textit{log_softmax}(\textbf{z}_u) \right] + \beta \frac{1}{2N_u}\left[-\sum_i\left(\log\sigma_i^2 + 1\right) + \sum_i\sigma_i^2 + \sum_i\mu^2_i\right] \hspace{5cm} (10)
\end{equation*}
$$

The authors proposed to use $\beta < 1$. If you remember, I previously mentioned that maximizing the likelihood involved minimizing $D_{KL}$. Since $D_{KL}$ measures the dissimilarity of the approximate posterior and the true posterior, by using $\beta < 1$ we are less able to predict "accurately" future user histories from historical data. In other words, we are potentially harming the loss. However, as I have discussed in some other posts/notebooks, when we build recommendation systems we are often not interested in achieving the best loss, but the best ranking metric. With that in mind, the authors showed that using $\beta < 1$ leads to better Normalised Discounted Cumulative Gain (NDCG) and Recall (R) metrics. They call this *partially regularized multinomial variational autoencoder* or $\text{Mult-VAE}^{\text{PR}}$

And..."*that's it!*". After all this we are ready to move to the code. There are two comments to make as we move to the code. I have implemented the Mult-VAE using `Mxnet` and `Pytorch`. Given the fact that these two implementations are nearly identical, I will concentrate here only on the `Mxnet` implementation, please go to the repo if you want to have a look to the `Pytorch` one. Secondly, the paper also includes a Multinomial Denoising Autoencoder, referred as Mult-DAE. However, given its simplicity relative to the formulation of the Mult-VAE discussed here I will not discussed the math of the Mult-DAE and focus simply on the code.

### $\text{Mult-VAE}^{\text{PR}}$, the code

After the explanation in the section above you might expect the code to look rather complex. However, you might feel disappointed/pleased when you see how simple it really is. 

In the [original publications](https://arxiv.org/pdf/1802.05814.pdf) the authors used a one hidden layer MLP as generative model. There they say that deeper architectures do not improve the results. In notebook 04 I will show the results and we will see that they are actually right. I have run over a hundred experiments and I find that the set up described in the paper leads to the best results. With that it mind, let's first have a look the model $ I \rightarrow 600 \rightarrow 200 \rightarrow 600 \rightarrow I$, where $I$ is the total number of items: 

<p align="center">
  <img width="800" src="figures/multvae_arch.png">
</p>

Fig1. 

In code, the model in the figure above is:

In [2]:
from typing import List

from mxnet import autograd
from mxnet.gluon import nn, HybridBlock

class VAEEncoder(HybridBlock):
    def __init__(self, q_dims: List[int], dropout: List[float]):
        super().__init__()

        # last dim multiplied by two for the reparameterization trick
        q_dims_ = q_dims[:-1] + [q_dims[-1] * 2]
        with self.name_scope():
            self.q_layers = nn.HybridSequential(prefix="q_net")
            for p, inp, out in zip(dropout, q_dims_[:-1], q_dims_[1:]):
                self.q_layers.add(nn.Dropout(p))
                self.q_layers.add(nn.Dense(in_units=inp, units=out))

    def hybrid_forward(self, F, X):
        h = F.L2Normalization(X)
        for i, layer in enumerate(self.q_layers):
            h = layer(h)
            if i != len(self.q_layers) - 1:
                h = F.tanh(h)
            else:
                mu, logvar = F.split(h, axis=1, num_outputs=2)
        return mu, logvar

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Before I move on, let me mention (and appreciate) one of the many nice "little" things that `Mxnet`'s `Gluon` has to offer. You will notice the use of `HybridBlock` and the use of the input `F` (the backend) when we define the forward pass, or more precisely, the `hybrid_forward` pass. One could write a full post on the joys of `HybridBlocks` and how nicely and easily the guys that developed `Gluon` brought together the flexibility of imperative frameworks (i.e. `Pytorch`) and the speed of declarative frameworks (i.e. `Tensorflow`) together. If you want to learn the details go [here](https://gluon.mxnet.io/chapter07_distributed-learning/hybridize.html), but believe me, this is **FAST**. 

In [7]:
class Decoder(HybridBlock):
    def __init__(self, p_dims: List[int], dropout: List[float]):
        super().__init__()

        with self.name_scope():
            self.p_layers = nn.HybridSequential(prefix="p_net")
            for p, inp, out in zip(dropout, p_dims[:-1], p_dims[1:]):
                self.p_layers.add(nn.Dropout(p))
                self.p_layers.add(nn.Dense(in_units=inp, units=out))

    def hybrid_forward(self, F, X):
        h = X
        for i, layer in enumerate(self.p_layers):
            h = layer(h)
            if i != len(self.p_layers) - 1:
                h = F.tanh(h)
        return h

In [8]:
class MultiVAE(HybridBlock):
    def __init__(
        self,
        p_dims: List[int],
        dropout_enc: List[float],
        dropout_dec: List[float],
        q_dims: List[int] = None,
    ):
        super().__init__()

        self.encode = VAEEncoder(q_dims, dropout_enc)
        self.decode = Decoder(p_dims, dropout_dec)

    def hybrid_forward(self, F, X):
        mu, logvar = self.encode(X)
        if autograd.is_training():
            std = F.exp(0.5 * logvar)
            eps = F.random.normal_like(std)
            mu = (eps * std) + mu
        return self.decode(mu), mu, logvar

...and, that's it...wait! We need the loss. Let me bring Eq 10 again

$$
\begin{equation*}
\mathcal L(\textbf{x}_u, \phi,\theta) = - \frac{1}{N_u}\sum_i x_{ui} \left[ \textit{log_softmax}(\textbf{z}_u) \right] + \beta \frac{1}{2N_u}\left[-\sum_i\left(\log\sigma_i^2 + 1\right) + \sum_i\sigma_i^2 + \sum_i\mu^2_i\right] \hspace{5cm} (10)
\end{equation*}
$$

In code

In [6]:
def vae_loss_fn(inp, out, mu, logvar, anneal):
    # firt term
    neg_ll = -nd.mean(nd.sum(nd.log_softmax(out) * inp, -1))
    # second term without beta
    KLD = -0.5 * nd.mean(nd.sum(1 + logvar - nd.power(mu, 2) - nd.exp(logvar), axis=1))
    # "full" loss (anneal is beta)
    return neg_ll + anneal * KLD

As I mentioned before, in the paper the authors also use a Multinomial Denoising Autoencoder (Mult-DAE}. The architecture is identical to that of the $\text{Mult-VAE}^{\text{PR}}$ apart from the fact that there is no variational aspect here. Therefore, not $D_{KL}$ regularization or reparameterization trick. The `Decoder` is identical to that shown above. The Encoder is as simple as:

In [9]:
class DAEEncoder(HybridBlock):
    def __init__(self, q_dims: List[int], dropout: List[float]):
        super().__init__()

        with self.name_scope():
            self.q_layers = nn.HybridSequential(prefix="q_net")
            for p, inp, out in zip(dropout, q_dims[:-1], q_dims[1:]):
                self.q_layers.add(nn.Dropout(p))
                self.q_layers.add(nn.Dense(in_units=inp, units=out))

    def hybrid_forward(self, F, X):
        h = F.L2Normalization(X)
        for layer in self.q_layers:
            h = F.tanh(layer(h))
        return h

and the model itself

In [10]:
class MultiDAE(HybridBlock):
    def __init__(
        self,
        p_dims: List[int],
        dropout_enc: List[float],
        dropout_dec: List[float],
        q_dims: List[int] = None,
    ):
        super().__init__()

        self.encode = DAEEncoder(q_dims, dropout_enc)
        self.decode = Decoder(p_dims, dropout_dec)

    def hybrid_forward(self, F, X):
        return self.decode(self.encode(X))

so, given the following parameters

In [23]:
I = 50000
q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]

In [24]:
vae_model = MultiVAE(
    p_dims=p_dims,
    q_dims=q_dims,
    dropout_enc=dropout_enc,
    dropout_dec=dropout_dec,
)

In [26]:
vae_model

MultiVAE(
  (encode): VAEEncoder(
    (q_layers): HybridSequential(
      (0): Dropout(p = 0.5, axes=())
      (1): Dense(50000 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 400, linear)
    )
  )
  (decode): Decoder(
    (p_layers): HybridSequential(
      (0): Dropout(p = 0.0, axes=())
      (1): Dense(200 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 50000, linear)
    )
  )
)

In [27]:
dae_model = MultiDAE(
    p_dims=p_dims,
    q_dims=q_dims,
    dropout_enc=dropout_enc,
    dropout_dec=dropout_dec,
)

In [28]:
dae_model

MultiDAE(
  (encode): DAEEncoder(
    (q_layers): HybridSequential(
      (0): Dropout(p = 0.5, axes=())
      (1): Dense(50000 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 200, linear)
    )
  )
  (decode): Decoder(
    (p_layers): HybridSequential(
      (0): Dropout(p = 0.0, axes=())
      (1): Dense(200 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 50000, linear)
    )
  )
)

And with all this, we can now move to notebook 03 and see how we can train and evaluate the model